In [ ]:
import pandas as pd
import numpy as np

a = pd.read_csv("mydbvscontigs.txt", sep='\t', engine="python",header=None)
a

In [ ]:
a.columns=["Contigs","Ref","Identity","Alignment_lenght","Mismatches","Gap_opens","q.start","q.end","s.start","s.end","evalue","bitscore"]
del a["Mismatches"]
del a["Gap_opens"]
del a["q.start"]
del a["q.end"]
del a["evalue"]
del a["bitscore"]
a

In [ ]:
b=a["Contigs"].str.split("_")

In [ ]:
c=[int(x[3]) for x in b]

In [ ]:
a.insert(4,"contig_length",c)
a.insert(5,"name",[x[1] for x in b])
a

In [ ]:
a["strand"] = ""

for i in range(a["s.start"].count()):
    if a["s.start"][i]<a["s.end"][i]:
        a["strand"][i]= "+"
    else:
        tmp = a["s.start"][i]
        a["s.start"][i] = a["s.end"][i]
        a["s.end"][i] = tmp
        a["strand"][i]= "-"
    
a   

In [ ]:
(a["Alignment_lenght"]/a["contig_length"]>=0.8).value_counts()

In [ ]:
d=a[a["Alignment_lenght"]/a["contig_length"]>=0.8]
d

In [ ]:
e=d["Ref"].value_counts()

In [ ]:
e.index

In [ ]:
d["Ref"].drop_duplicates()

In [ ]:
e=d[d["Ref"]=="MF474175.1"].sort_values(by=['s.start'])
e

In [ ]:
previous = "0"
for i in range(1, e["s.start"].count()):
    first=e.iloc[i-1]
    second=e.iloc[i]
    if first["s.end"]<second["s.start"]:
        # no overlap
        print(first["name"],"->", second["name"])
    else:
        print(previous,"->", second["name"])
    previous = second["name"]

In [ ]:
LR025097=d[d["Ref"]=="LR025097.1"].sort_values(by=['s.start'])
LR025097

In [ ]:
KJ484628=d[d["Ref"]=="KJ484628.1"].sort_values(by=['s.start'])
KJ484628

In [ ]:
KJ484626 = d[d["Ref"]=="KJ484626.1"].sort_values(by=['s.start'])
KJ484626

In [ ]:
CP029744 = d[d["Ref"]=="CP029744.1"].sort_values(by=['s.start'])
CP029744

In [ ]:
CP027357 = d[d["Ref"]=="CP027357.1"].sort_values(by=['s.start'])
CP027357

In [ ]:
CP021728 = d[d["Ref"]=="CP021728.1"].sort_values(by=['s.start'])
CP021728

In [ ]:
CP019007 = d[d["Ref"]=="CP019007.1"].sort_values(by=['s.start'])
CP019007

In [ ]:
AF550679 = d[d["Ref"]=="AF550679.1"].sort_values(by=['s.start'])
AF550679

In [ ]:
KJ170699 = d[d["Ref"]=="KJ170699.1"].sort_values(by=['s.start'])
KJ170699

In [ ]:
import graphviz

def drawing_graphviz(ref, label="", N=8):
    g=graphviz.Digraph()
    g.attr(rankdir='LR', label=label)

    seen={}
    def edge(x,y):
        if seen.get((x,y), 0) == 0:
            g.edge(x, y)
            seen[ (x,y) ] = 1

    sg = {}
    for i in range(N):
        sg[i] = graphviz.Digraph()
        sg[i].attr(rank="same")

    second=ref.iloc[0]
    if(second["strand"]=="+"):
        g.node(second["name"]+str(second["s.start"]), second["name"], shape='rarrow')
        sg[0].node(second["name"]+str(second["s.start"]), second["name"], shape='rarrow')
    else:
        g.node(second["name"]+str(second["s.start"]), second["name"], shape='larrow')
        sg[0].node(second["name"]+str(second["s.start"]), second["name"], shape='larrow')

    previous = second["name"]
    prev_strand = str(second["s.start"])

    for i in range(1, ref["s.start"].count()):
        first=ref.iloc[i-1]
        second=ref.iloc[i]
        if(second["strand"]=="+"):
            g.node(second["name"]+str(second["s.start"]), second["name"], shape='rarrow')
            sg[i%N].node(second["name"]+str(second["s.start"]), second["name"], shape='rarrow')
        else:
            g.node(second["name"]+str(second["s.start"]), second["name"], shape='larrow')
            sg[i%N].node(second["name"]+str(second["s.start"]), second["name"], shape='larrow')
        if first["s.end"]<second["s.start"]:
            # no overlap
            edge(first["name"]+str(first["s.start"]), second["name"]+str(second["s.start"]))
        else:
            edge(previous+prev_strand, second["name"]+str(second["s.start"]))
        previous = second["name"]
        prev_strand = str(second["s.start"])

    for i in range(N):
        g.subgraph(sg[i])

    return(g)

In [ ]:
drawing_graphviz(LR025097,"LR025097").render("LR025097.gv","Graphviz/","pdf")
drawing_graphviz(KJ484628,"KJ484628").render("KJ484628.gv","Graphviz/","pdf")
drawing_graphviz(KJ484626,"KJ484626").render("KJ484626.gv","Graphviz/","pdf")
drawing_graphviz(CP029744,"CP029744").render("CP029744.gv","Graphviz/","pdf")
drawing_graphviz(CP027357,"CP027357").render("CP027357.gv","Graphviz/","pdf")
drawing_graphviz(CP021728,"CP021728").render("CP021728.gv","Graphviz/","pdf")
drawing_graphviz(CP019007,"CP019007").render("CP019007.gv","Graphviz/","pdf")
drawing_graphviz(AF550679,"AF550679").render("AF550679.gv","Graphviz/","pdf")
drawing_graphviz(KJ170699,"KJ170699").render("KJ170699.gv","Graphviz/","pdf")
drawing_graphviz(e,"MF474175").render("MF474175.gv","Graphviz/","pdf")

In [ ]:
def col2list(dataframe,col):
    return dataframe[col].to_list()

In [ ]:
LR025097_list = col2list(LR025097, "name")
KJ484628_list = col2list(KJ484628, "name")
len(LR025097_list),len(KJ484628_list)

In [ ]:
matrix = pd.DataFrame(index=KJ484628_list, columns=LR025097_list)

for i in matrix.index:
    for j in matrix.columns:
        if i==j:
            matrix.loc[i,j]=1
        else:
            matrix.loc[i,j]=0

matrix